In [75]:
import cv2
import numpy as np
import os

In [76]:
sequence_folder = 'sequence/'
frame_files = sorted([f for f in os.listdir(sequence_folder) if f.endswith('.jpg') or f.endswith('.png')])
initial_bbox = (246, 208, 23, 23) 

initial_hypothesis = np.array([initial_bbox[1], initial_bbox[0], 
                                   initial_bbox[1] + initial_bbox[3], 
                                   initial_bbox[0] + initial_bbox[2]])
frames = (cv2.imread(os.path.join(sequence_folder, f)) for f in frame_files)
frame = list(frames)[0]

In [77]:
top, left, bottom, right = initial_hypothesis

In [78]:
object_mask = np.zeros(frame.shape[:2], dtype=bool)
object_mask[top:bottom, left:right] = True


In [79]:
padding = 20
surrounding_top = max(0, top - padding)
surrounding_left = max(0, left - padding)
surrounding_bottom = min(frame.shape[0], bottom + padding)
surrounding_right = min(frame.shape[1], right + padding)
surrounding_mask = np.zeros(frame.shape[:2], dtype=bool)
surrounding_mask[surrounding_top:surrounding_bottom, surrounding_left:surrounding_right] = True
surrounding_mask[top:bottom, left:right] = False  # exclude the object region

In [80]:
def compute_histogram(image, region_mask, bins):
    histogram = np.zeros(bins)
    # histogram = cv2.calcHist([image], [0,1,2], region_mask.astype(np.uint8), [bins,bins,bins], [0, 256,0, 256,0, 256,0, 256]).flatten()
    print(histogram.shape)

    for c in range(image.shape[-1]):  
        hist =cv2.calcHist([image], [c], region_mask.astype(np.uint8), [bins], [0, 256]).flatten()
        histogram += hist
    return histogram
    # histogram = cv2.calcHist([image], [0], region_mask.astype(np.uint8), [bins], [0, 256]).flatten()

In [81]:
bins = 10

In [82]:
H_O = compute_histogram(frame, object_mask, bins) 
H_S = compute_histogram(frame, surrounding_mask, bins)


(10,)
(10,)


In [83]:
probabilities = np.zeros(frame.shape[:2])
probabilities.shape

(360, 640)

In [84]:

def calculateProbabilities(I_x, H_O, H_S):

    H_O_b = H_O[I_x] 
    H_S_b = H_S[I_x] 

    total_hist = H_O_b + H_S_b

    return H_O_b / total_hist


In [85]:
def get_bin_indices(pixel_values, bins, value_range=256):
    return np.array([int((pixel_value * bins) // value_range) for pixel_value in pixel_values])


In [86]:
for y in range(frame.shape[0]):
    for x in range(frame.shape[1]):
            pixel_values = frame[y, x]
            bin_index = int(get_bin_indices(pixel_values, bins).mean())
            if surrounding_mask[y, x] or object_mask[y, x]:
                probabilities[y, x] = calculateProbabilities(bin_index, H_O, H_S)
            else:
                probabilities[y, x] = 0.5



In [ ]:
cv2.rectangle(probabilities, (left, top), (right, bottom), (0, 255, 0), 2)
cv2.imshow('Object Tracking', probabilities)
key = cv2.waitKey(0) & 0xFF
cv2.destroyAllWindows()